In [70]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sklearn


In [2]:
import keras

Using CNTK backend
e:\Anaconda3\envs\cntk-py36\lib\site-packages\keras\backend\cntk_backend.py:19: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


## COMP 3132 -  Machine Learning with Python
__Week 3 lecture__

### [github.com/kamrik/ML2](https://github.com/kamrik/ML2)

### [slido.com/COMP3132](http://slido.com/COMP3132)

## Plan for today
 - Classification of IMDB reviews, line by line
   - Start with sklearn
   - Review model evaluation techniques
   - Classification using Keras
 

## Home assignment 1 is due tomorrow night

## Classifying movie reviews as positive or negative
 * Back on Nov 13th we classified Yelp reviews - [ML1/exercises/yelp.ipynb](https://github.com/kamrik/ML1/blob/master/exercises/yelp.ipynb) (see [solutions](https://nbviewer.jupyter.org/url/kamrik.org/ML1/yelp_solutions.ipynb))

## Preprocessing - representing the text

In [5]:
from keras.datasets import imdb

In [6]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [13]:
words = imdb.get_word_index()

## Explore the data

In [17]:
type(words) 

dict

## Where the data comes from
 - Does it get cached?

In [18]:
imdb.load_data()
imdb.get_word_index()

## What's that `npz` file?
https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.savez.html

## Can we read the text?

In [36]:
# How do we convert numbers back to text given the dictionary?

## Reversed dict

In [22]:
num2word = {num:word for word, num in words.items()}

In [44]:
[num2word[i] for i in range(1,10)]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it']

In [39]:
nums = train_data[5]
print(nums)

[1, 778, 128, 74, 12, 630, 163, 15, 4, 1766, 7982, 1051, 2, 32, 85, 156, 45, 40, 148, 139, 121, 664, 665, 10, 10, 1361, 173, 4, 749, 2, 16, 3804, 8, 4, 226, 65, 12, 43, 127, 24, 2, 10, 10]


In [40]:
txt = [num2word.get(num - 3, '?') for num in nums]
print(txt)

['?', 'begins', 'better', 'than', 'it', 'ends', 'funny', 'that', 'the', 'russian', 'submarine', 'crew', '?', 'all', 'other', 'actors', "it's", 'like', 'those', 'scenes', 'where', 'documentary', 'shots', 'br', 'br', 'spoiler', 'part', 'the', 'message', '?', 'was', 'contrary', 'to', 'the', 'whole', 'story', 'it', 'just', 'does', 'not', '?', 'br', 'br']


## One-hot encoding

In [47]:
N = 10000
vec = np.zeros(N, dtype='float32')
vec

# Set the words that are present to 1

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [48]:
def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension), dtype='float32')
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [50]:
x_test

array([[ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.]], dtype=float32)

## Use sklearn

In [54]:
# Import
from sklearn.naive_bayes import MultinomialNB

In [55]:
# Instantiate
clf = MultinomialNB()

In [56]:
# Fit
clf.fit(x_train, train_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [57]:
# Predict
x_pred = clf.predict(x_test)

## Find model accuracy

In [62]:
x_pred == test_labels

array([ True, False,  True, ..., False,  True, False], dtype=bool)

## Let's try the same with neural nets

In [63]:
from keras import models
from keras import layers

## How many layers? How many neurons?
 - Output is probability of positive

In [64]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [66]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [71]:
t = time.time()
history = model.fit(x_train,
                    train_labels,
                    epochs=10,
                    batch_size=512,
                    validation_data=(x_test, test_labels))
print("Elapsed: ", time.time() - t)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s - loss: 0.0132 - acc: 0.9971 - val_loss: 0.7564 - val_acc: 0.8539
Epoch 2/10
25000/25000 [==============================] - 6s - loss: 0.0112 - acc: 0.9977 - val_loss: 0.7878 - val_acc: 0.8522
Epoch 3/10
25000/25000 [==============================] - 6s - loss: 0.0109 - acc: 0.9976 - val_loss: 0.8173 - val_acc: 0.8528
Epoch 4/10
25000/25000 [==============================] - 6s - loss: 0.0064 - acc: 0.9990 - val_loss: 0.8323 - val_acc: 0.8497
Epoch 5/10
25000/25000 [==============================] - 6s - loss: 0.0094 - acc: 0.9979 - val_loss: 0.8950 - val_acc: 0.85181s - loss: 0.0
Epoch 6/10
25000/25000 [==============================] - 6s - loss: 0.0083 - acc: 0.9980 - val_loss: 0.9357 - val_acc: 0.8496
Epoch 7/10
25000/25000 [==============================] - 6s - loss: 0.0041 - acc: 0.9992 - val_loss: 0.9358 - val_acc: 0.8491
Epoch 8/10
25000/25000 [=======================

## Train / Validation / Test